# Домашнее задание 2
Для работы я написала свой текст (на 207 слов) и разметила его вручную. Мини-корпус лежит в файле *gold.tsv*. Для рразметки я пользовалась придуманным мною тэгсэтам, основанным на тэгсэте pymorphy.
Дальше мною были выбраны три морфо-парсера (pymorphy, mystem и slovnet, который является частью natasha)

In [112]:
import csv
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
import pandas as pd
from sklearn.metrics import accuracy_score

In [113]:
from razdel import sentenize, tokenize
from navec import Navec
from slovnet import Morph

In [114]:
textlist = []
gold_pos = []
with open('gold.tsv', 'r', encoding='utf-8') as data:
    tsv_file = csv.reader(data, delimiter="\t")
    for line in tsv_file:
        textlist.append(line[0])
        gold_pos.append(line[1])

In [115]:
from pymystem3 import Mystem
m = Mystem()

In [116]:
ana = m.analyze(' '.join(textlist))

С mystem возникали некоторые проблемы, потому что он принимает на вход текст, а не список слов, пришлось дорабатывать некоторые случаи вручную

In [134]:
mystem_pos = []
mst = []
for word in ana:
    if 'analysis' in word:
        if len(word['analysis']) > 0:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            mystem_pos.append(pos)
            mst.append([word['text'], pos])
        elif word['text'] != ' ' and word['text'] != '00' and word['text'] != ':' and word['text'] != '\n':
            pos = 'UNKN'
            mystem_pos.append(pos)
            mst.append([word['text'], pos])
    elif word['text'] != ' ' and word['text'] != '00' and word['text'] != ':' and word['text'] != '\n':
            pos = 'UNKN'
            mystem_pos.append(pos)
            mst.append([word['text'], pos])

In [121]:
pymorphy_pos = []
for word in textlist:
    pym = morph.parse(word)
    tags = str(pym[0].tag).split(',')
    pos = tags[0]
    if len(pos) > 4:
        new_tags = pos.split(' ')
        pos = new_tags[0]
    pymorphy_pos.append(pos)

In [123]:
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
morphy = Morph.load('slovnet_morph_news_v1.tar', batch_size=4)
morphy.navec(navec)

slovnet_pos = []
chunk = []
chunk.append(textlist)
markup = next(morphy.map(chunk))
for token in markup.tokens:
    POS = token.tag.split('|')[0]
    slovnet_pos.append(POS)

После обработки данных, соберем их все в датафрейм и пропишем функции, которые приведут все тэгсэты к общему виду

In [125]:
df = pd.DataFrame({'pos': textlist, 'gold': gold_pos, 'mystem': mystem_pos, 'pymorphy': pymorphy_pos, 'slovnet': slovnet_pos})

In [126]:
def pmtogold(PartOfSpeech):
    adj = ['ADJF', 'ADJS', 'COMP']
    verb = ['VERB', 'INFN', 'PRTF', 'PRTS', 'GRND']
    if PartOfSpeech in adj:
        return 'ADJ'
    elif PartOfSpeech in verb:
        return 'VERB'
    elif PartOfSpeech == 'NPRO':
        return 'PRON'
    elif PartOfSpeech == 'PRCL':
        return 'PART'
    else:
        return PartOfSpeech

In [127]:
def mstogold(PartOfSpeech):
    if PartOfSpeech == 'PR':
        return 'PREP'
    elif PartOfSpeech == 'A':
        return 'ADJ'
    elif PartOfSpeech == 'S':
        return 'NOUN'
    elif PartOfSpeech == 'ADV':
        return 'ADVB'
    elif PartOfSpeech == 'ADVPRO' or PartOfSpeech == 'APRO' or PartOfSpeech == 'SPRO':
        return 'PRON'
    elif PartOfSpeech == 'ANUM' or PartOfSpeech == 'NUM':
        return 'NUMR'
    elif PartOfSpeech == 'V':
        return 'VERB'
    else:
        return PartOfSpeech

In [128]:
def sntogold(PartOfSpeech):
    if PartOfSpeech == 'DET':
        return 'PRON'
    elif PartOfSpeech == 'ADP':
        return 'PREP'
    elif PartOfSpeech == 'NUM':
        return 'NUMR'
    elif PartOfSpeech == 'CCONJ' or PartOfSpeech == 'SCONJ':
        return 'CONJ'
    elif PartOfSpeech == 'ADV':
        return 'ADVB'
    elif PartOfSpeech == 'PROPN':
        return 'NOUN'
    elif PartOfSpeech == 'AUX':
        return 'VERB'
    else:
        return PartOfSpeech

In [129]:
df['pymorphy'] = df['pymorphy'].apply(pmtogold)
df['mystem'] = df['mystem'].apply(mstogold)
df['slovnet'] = df['slovnet'].apply(sntogold)

In [130]:
df

,pos,gold,mystem,pymorphy,slovnet
0,наш,PRON,PRON,ADJ,PRON
1,12часовой,ADJ,UNKN,ADJ,ADJ
2,сон,NOUN,NOUN,NOUN,NOUN
3,плохо,ADVB,ADVB,PRED,ADJ
4,для,PREP,PREP,PREP,PREP
...,...,...,...,...,...
202,хорошего,ADJ,ADJ,ADJ,ADJ
203,придумать,VERB,VERB,VERB,VERB
204,не,PART,PART,PART,PART
205,могу,VERB,VERB,VERB,VERB


In [131]:
gold = df['gold'].tolist()
mystem = df['mystem'].tolist()
pymorphy = df['pymorphy'].tolist()
slovnet = df['slovnet'].tolist()

Выведем точность каждого из трёх парсеров:

In [132]:
print('Точность mystem: ', accuracy_score(mystem, gold))
print('Точность pymorphy: ', accuracy_score(pymorphy, gold))
print('Точность slovnet: ', accuracy_score(slovnet, gold))

Точность mystem:  0.8695652173913043
Точность pymorphy:  0.8019323671497585
Точность slovnet:  0.855072463768116


In [137]:
mst

[['наш', 'APRO'],
 ['12часовой', 'UNKN'],
 ['сон', 'S'],
 ['плохо', 'ADV'],
 ['для', 'PR'],
 ['нас', 'SPRO'],
 ['кончился', 'V'],
 ['получился', 'V'],
 ['полный', 'A'],
 ['кринж', 'S'],
 ['c', 'UNKN'],
 ['утра', 'S'],
 ['мне', 'SPRO'],
 ['пришлось', 'V'],
 ['пить', 'V'],
 ['кофе', 'S'],
 ['и', 'CONJ'],
 ['накидывать', 'V'],
 ['пальто', 'S'],
 ['за', 'PR'],
 ['пять', 'NUM'],
 ['минут', 'S'],
 ['при', 'PR'],
 ['этом', 'SPRO'],
 ['все', 'APRO'],
 ['мои', 'APRO'],
 ['попытки', 'S'],
 ['успеть', 'V'],
 ['на', 'PR'],
 ['автобус', 'S'],
 ['оказались', 'V'],
 ['тщетны', 'A'],
 ['вызвала', 'V'],
 ['такси', 'S'],
 ['но', 'CONJ'],
 ['из-за', 'PR'],
 ['пробок', 'S'],
 ['всё', 'SPRO'],
 ['равно', 'A'],
 ['опоздала', 'V'],
 ['на', 'PR'],
 ['электричку', 'S'],
 ['я', 'SPRO'],
 ['думала', 'V'],
 ['на', 'PR'],
 ['работе', 'S'],
 ['меня', 'SPRO'],
 ['казнят', 'V'],
 ['за', 'PR'],
 ['такое', 'APRO'],
 ['в', 'PR'],
 ['то', 'APRO'],
 ['же', 'PART'],
 ['время', 'S'],
 ['моя', 'APRO'],
 ['подруга', 'S'],
 ['

Таким образом, самым точным парсером оказался *mystem*, дальше будем пользоваться им. 
Для начала напишем функцию chunker - я делаю ее таким образом, чтобы можно было ввести две любые части речи и получить последовательность из двух слов

In [149]:
def chunker(pos1, pos2):
    combs = []
    for combination in mst:
        if combination[1] == pos1:
            if mst[mst.index(combination) + 1][1] == pos2:
                combs.append(str(combination[0] + ' ' + mst[mst.index(combination) + 1][0]))
    return combs

In [150]:
noun_adverb = chunker('S', 'ADV')
print(noun_adverb)

['сон плохо', 'подруга всё', 'твита конечно', 'жизни обязательно', 'жизни обязательно']


In [151]:
verb_noun = chunker('V', 'S')
print(verb_noun)

['пить кофе', 'накидывать пальто', 'вызвала такси', 'принесла бутылку', 'посмотреть Хор', 'радоваться жизни', 'пить кофе']


In [160]:
verb_verb = chunker('V', 'V')
print(verb_verb)

['кончился получился', 'пришлось пить', 'спешила позавтракала', 'начали убеждать', 'убеждать проголосовать', 'решили посмотреть', 'брошу пить', 'малясь шатаясь']


Пункт 5. Кажется, что для определения тональности было бы хорошо добавить сочетания 'не' с прилагательными. Кажется, они будут часто встречаться в отрицательных отзывах. То же самое с 'не' и глаголами. Обычно отрицания используются при описании недостатков.
Ещё было бы интересно посомтреть и в положительных, и в отрицательных отзывах на сочетание наречие + прилагательное. Где больше ярких эмоций, выраженных через "очень плохой"/"очень хороший"

//код в предыдущую домашку вписать я не успела